# **📚 RAG-Based Academic Answering System**
### *Retrieval-Augmented Generation for Exam-Oriented, Reference-Based Answers*

---

### **✅ Introduction**

In today’s academic world, AI tools like **ChatGPT** have become very common for learning and answer writing. But a serious issue has emerged: many students directly copy AI-generated answers **without syllabus alignment, without references, and sometimes with incorrect/irrelevant content**.  

This became clearly visible in our **3rd semester exams**, where some students wrote answers using ChatGPT, and teachers **did not award marks**. In fact, it was clearly stated that answers written directly from ChatGPT would not be accepted.

---

### **🎯 Goal of This Project**

To solve this problem, this notebook focuses on building a **RAG (Retrieval-Augmented Generation) system** — an AI assistant that generates answers **only after retrieving content from trusted academic sources** such as:

- 📄 Teacher-provided notes  
- 📘 Textbooks  
- 🗂️ College PDFs / Question banks  
- 📝 Syllabus-oriented study material  

Instead of giving generic internet-based answers, this system produces **document-grounded responses** that are more reliable and exam-appropriate.

---

### **🌟 Why RAG?**

A standard chatbot may generate fluent answers, but it can:
- ❌ hallucinate (give wrong information)
- ❌ ignore the syllabus
- ❌ provide vague or over-general responses
- ❌ lack sources/references

A **RAG-based system** improves this by ensuring:

✅ **Syllabus-oriented answers**  
✅ **Trustworthy and factual content**  
✅ **Reference-backed generation**  
✅ **Less hallucination / higher accuracy**  
✅ **Teacher-acceptable content**  

---

### **🧠 What This Notebook Implements**

This notebook builds a complete **RAG Pipeline**, step-by-step:

1. 📥 Load academic documents (PDF/notes/text)
2. ✂️ Split text into meaningful chunks
3. 🧾 Generate embeddings for semantic search
4. 🗃️ Store embeddings into a vector database
5. 🔎 Retrieve top relevant chunks for a query
6. 🤖 Generate final answer using LLM grounded in retrieved context

---

### **🏁 Outcome**

At the end of this notebook, we will have an **Academic RAG Assistant** that can generate:

📌 **Exam-focused answers**  
📌 **Reliable content from given documents**  
📌 **Reference-supported responses**  

This makes the output more authentic, more accurate, and suitable for academic evaluation.

---


#### **Reading .pdf File from Directory**

In [4]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader


dir_loader = DirectoryLoader(
    path=r"C:\Users\sujal warghe\Desktop\sample_project_1\RAG_Dev\data\pdf",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=True
)

pdf_documents = dir_loader.load()
print(len(pdf_documents))


c:\Users\sujal warghe\Desktop\sample_project_1\RAG_Dev\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 6/6 [00:01<00:00,  4.03it/s]

456


### **🔑 API Configuration & Environment Variables**

This notebook uses the following external APIs loaded from `.env` file:
1. **GROQ API** - For LLM (Language Model) operations using Llama-3.1
2. **Typesense API** - For advanced document search and retrieval
3. **Sentence Transformers** - For generating embeddings (local, no API key needed)
4. **ChromaDB** - For vector storage (local database)

All API keys should be stored in `.env` file (not committed to version control).

 #### **Creating Embedding Manager** 

##### **importing ....**

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

##### In this notebook we are creating embedding manager from scratch

#### code me __init__() ka role :
* Model name set krna:
>self.model_name = model_name

* Model variable create karna:
>self.model = None

* Model automatically load karna:
>self._load_model()


In [6]:
class EmbeddingManager:

    def __init__(self, model_name:str = 'all-MiniLM-L6-v2'):

        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):

        try:
            print(f"Loading Embedding Model...{self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")

        except Exception as e:
            print(f"Error Loading Model.. {self.model_name} : {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model is not Loaded...")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

    def get_embedding_dimension(self) -> int:

        if not self.model:
            raise ValueError("Model not loaded..")
        return self.model.get_sentence_embedding_dimension()

# initializing embedding manager

embedding_manager = EmbeddingManager()
    

Loading Embedding Model...all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    split_docs = splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    return split_docs


In [8]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
pdf_loader = PyPDFDirectoryLoader(r"C:\Users\sujal warghe\Desktop\sample_project_1\RAG_Dev\data\pdf")
all_pdf_documents = pdf_loader.load()
chunks = split_documents(all_pdf_documents)
chunks

Split 456 documents into 492 chunks


[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2023-11-09T20:20:56+05:30', 'author': 'prathvi kumari', 'moddate': '2023-11-09T20:20:56+05:30', 'source': 'C:\\Users\\sujal warghe\\Desktop\\sample_project_1\\RAG_Dev\\data\\pdf\\Introduction to Sensors n Actuators.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content='MODULE 2: SENSORS, TRANSDUCERS, AND ACTUATORS \n \n1 \n                                                                                                                                   MITE \nSensors - Introduction, Classification of sensors with examples, Characteristics, Construction \nand working of Photo -resistive sensor and Thermistor, their applications (qualitative). \nNumerical Problems.  \nTransducers: Introduction to Transducers, Types of Transducers. Principle, construction and \nworking of Linear Variable Differential Transformer, Difference  between sensor and \ntransducer \nActuators:

In [9]:
import os

In [10]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = r"C:\Users\sujal warghe\Desktop\sample_project_1\RAG_Dev\data\vector_store"):
        """
        intialize the vector store
        
        Args:
           collection_name: Name of the ChromaDB collection
           persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_direcory = persist_directory
        self.client = None
        self.collection = None
        self.initialize_store()

    def initialize_store(self):
        """
        initialize chromaDB client and collection
        """
        try:
            # Create persistant ChromaDB client
            os.makedirs(self.persist_direcory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_direcory)

            # Get or create collections
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """ 
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of Langchain documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents to vector of embeddings")
        
        print(f"Adding {len(documents)} documents must match of embeddings")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 847


In [11]:
### convert the text  to embeddings
texts = [doc.page_content for doc in chunks]

## Generate the embeddings

embeddings = embedding_manager.generate_embeddings(texts)

## store int he vector database
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 492 texts...


Batches: 100%|██████████| 16/16 [00:18<00:00,  1.17s/it]


Generated embeddings with shape: (492, 384)
Adding 492 documents must match of embeddings
Successfully added 492 documents to vector store
Total documents in collection: 1339


In [12]:
class RAGRetriever:
    """Handles query-based retrieval"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retirever

        Args:
           vector_store: vector containing document embeddings
           embedding_manager for generating query embeddings 
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
       Retrieve relevant documents for a query

       Args:
          query: The search query
          top_k: Number of top results to return
          score_threshold: Minimum similarity score threshold

       Return:
           List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: {query}")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id' : doc_id,
                            'content' : document,
                            'metadata' : metadata,
                            'similarity_score' : similarity_score,
                            'distance' : distance,
                            'rank' : i + 1
                        })

                print(f'Retrieved {len(retrieved_docs)} documents (after filtering)')
            else:
                print("No documents found")

            return retrieved_docs
        except Exception as e:
            print(f'Error during retrieval: {e}')
            return []
        
rag_retriever = RAGRetriever(vectorstore, embedding_manager)




In [13]:
rag_retriever.retrieve("what is accuracy and precision ?")

Retrieving documents for query: what is accuracy and precision ?
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 44.54it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_63eb74bd_99',
  'content': '06-01-2026MDM_SENSORS_ACTUATORS 48\nSENSORS CHARACTERISTICS\n❑ ERROR: IT IS THE DIFFERENCE BETWEEN THE RESULT OF THE MEASUREMENT AND THE TRUE \nVALUE OF THE QUANTITY BEING MEASURED. THE CLASSIFICATION OF ERRORS ARE AS \nFOLLOWS:  ABSOLUTE ERRORS & RELATIVE ERROR \n ABSOLUTE ERRORS = MEASURED VALUE – TRUE VALUE \n RELATIVE ERROR = ABSOLUTE ERRORS / TRUE VALUE \n❑ ACCURACY: IT IS THE DIFFERENCE BETWEEN THE MEASURED VALUE AND TRUE VALUE. THE \nACCURACY DEFINES THE CLOSENESS BETWEEN THE ACTUAL MEASURED VALUE AND A TRUE \nVALUE. \n❑ PRECISION: PRECISION IS THE ABILITY TO REPRODUCE REPEATEDLY WITH A GIVEN \nACCURACY.',
  'metadata': {'source': 'C:\\Users\\sujal warghe\\Desktop\\sample_project_1\\RAG_Dev\\data\\pdf\\MDM_SENSORS_ACTUATORS_U1.pptx.pdf',
   'doc_index': 99,
   'content_length': 607,
   'creator': 'Google',
   'creationdate': '',
   'page_label': '48',
   'total_pages': 69,
   'producer': 'PyPDF',
   'page': 47,
   'title': 'MDM_SENSORS_AC

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads .env into environment

# Load all API keys from .env
groq_api_key = os.getenv("GROQ_API_KEY")
typesense_api_key = os.getenv("TYPESENSE_API_KEY")
typesense_host = os.getenv("TYPESENSE_HOST")
typesense_port = os.getenv("TYPESENSE_PORT")
typesense_protocol = os.getenv("TYPESENSE_PROTOCOL")

# Validate GROQ API
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in .env file")

# Validate Typesense APIs
if not all([typesense_api_key, typesense_host, typesense_port, typesense_protocol]):
    raise ValueError("Typesense configuration incomplete in .env file")

In [15]:
# ✅ Display All Configured APIs
print("\n" + "="*80)
print("📋 RAG SYSTEM - API & ENVIRONMENT CONFIGURATION SUMMARY")
print("="*80)

api_config = {
    "GROQ_API_KEY": {
        "status": "✅ LOADED" if groq_api_key else "❌ NOT FOUND",
        "description": "LLM API for Llama-3.1 model",
        "usage": "Generate contextual answers",
        "model": "llama-3.1-8b-instant"
    },
    "TYPESENSE_API": {
        "status": "✅ CONFIGURED",
        "description": "Vector search and document retrieval",
        "usage": "Advanced semantic search",
        "host": "4rcjogtule2hxs8dp-1.a1.typesense.net"
    },
    "SENTENCE_TRANSFORMERS": {
        "status": "✅ LOCAL",
        "description": "Embedding generation model",
        "usage": "Convert text to vectors",
        "model": "Sentence Transformer (384-dim)"
    },
    "CHROMADB": {
        "status": "✅ LOCAL",
        "description": "Vector database",
        "usage": "Store and retrieve document embeddings",
        "location": "data/vector_store/"
    }
}

for api_name, config in api_config.items():
    print(f"\n{api_name}")
    for key, value in config.items():
        print(f"  • {key.replace('_', ' ').title()}: {value}")

print("\n" + "="*80)
print("✅ ALL APIS CONFIGURED AND READY")
print("="*80 + "\n")


📋 RAG SYSTEM - API & ENVIRONMENT CONFIGURATION SUMMARY

GROQ_API_KEY
  • Status: ✅ LOADED
  • Description: LLM API for Llama-3.1 model
  • Usage: Generate contextual answers
  • Model: llama-3.1-8b-instant

TYPESENSE_API
  • Status: ✅ CONFIGURED
  • Description: Vector search and document retrieval
  • Usage: Advanced semantic search
  • Host: 4rcjogtule2hxs8dp-1.a1.typesense.net

SENTENCE_TRANSFORMERS
  • Status: ✅ LOCAL
  • Description: Embedding generation model
  • Usage: Convert text to vectors
  • Model: Sentence Transformer (384-dim)

CHROMADB
  • Status: ✅ LOCAL
  • Description: Vector database
  • Usage: Store and retrieve document embeddings
  • Location: data/vector_store/

✅ ALL APIS CONFIGURED AND READY



In [16]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-8b-instant", temperature=0.1, max_tokens=1024)

def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer' : 'No relevent context found.', 'sources' : [], 'confidence' : 0.0, 'context' : ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source' : doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page' : doc['metadata'].get('page', 'unknown'),
        'score' : doc['similarity_score'],
        'preview' : doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answwer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer' : response.content,
        'source' : sources,
        'confidence' : confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is accuracy and precision in depth?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['source'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: What is accuracy and precision in depth?
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 103.71it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Answer: Accuracy and precision are two fundamental concepts in measurement and sensing.

**Accuracy** refers to how close a measurement is to the true value. It's a measure of how close the measured value is to the actual value. Think of it as hitting a target - accuracy is how close you are to the bullseye.

**Precision**, on the other hand, refers to the ability to reproduce the same measurement repeatedly under the same conditions. It's a measure of how consistent the measurements are. Think of it as throwing darts at the same target - precision is how close each dart is to the others.

To illustrate the difference:

* A measurement that is accurate but not precise might be close to the true value, but the results vary widely each time you take the measurement.
* A measurement that is precise but not accurate might be consistent, but consistently off from the true value.

In summary, accuracy is about being close to the truth, while precision is about being consistent in your measur

In [17]:
import typesense

In [ ]:
client = typesense.Client({
    'nodes': [{
        'host': typesense_host,  # Loaded from .env
        'port': typesense_port,  # Loaded from .env
        'protocol': typesense_protocol  # Loaded from .env
    }],
    'api_key': typesense_api_key,  # Loaded from .env
    'connection_timeout_seconds': 2

})

# Creating schema for our json data
rag_schema = {
    "name": "MDM_RAG",
    "fields": [
        {"name": "id", "type": "string"},

        # main text chunk
        {"name": "content", "type": "string"},

        # metadata (citations)
        {"name": "source", "type": "string", "facet": True},
        {"name": "page", "type": "int32", "facet": True},

        # optional tags (filtering)
        {"name": "unit", "type": "string", "facet": True, "optional": True},
        {"name": "topic", "type": "string", "facet": True, "optional": True},
        {"name": "subject", "type": "string", "facet": True, "optional": True},

        # embedding vector
        {"name": "embedding", "type": "float[]", "num_dim": 384}
    ]
}

print(client.collections.create(rag_schema))


# After running go to that clusters tab >> go to >> collections
# there is one collection created as name books

In [19]:
client

### **Ingestion in Typesense Cloud**

In [21]:
import os
import uuid
import typesense
import fitz  # PyMuPDF
from dotenv import load_dotenv
from tqdm import tqdm

from langchain_huggingface import HuggingFaceEmbeddings


In [22]:
load_dotenv()

PDF_DIR = "data/pdf"

TYPESENSE_HOST = os.getenv("TYPESENSE_HOST")
TYPESENSE_PORT = os.getenv("TYPESENSE_PORT")
TYPESENSE_PROTOCOL = os.getenv("TYPESENSE_PROTOCOL", "http")
TYPESENSE_API_KEY = os.getenv("TYPESENSE_API_KEY")

In [23]:
# Initialize Embedding Model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [25]:
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


### **PDF → Typesense Ingestion**

In [26]:
PDF_DIR = "data/pdf"

for pdf_file in tqdm(os.listdir(PDF_DIR)):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(PDF_DIR, pdf_file)
    doc = fitz.open(pdf_path)

    for page_number in range(len(doc)):
        page = doc[page_number]
        text = page.get_text().strip()

        if not text:
            continue

        chunks = chunk_text(text)

        for chunk in chunks:
            embedding = embedding_model.embed_query(chunk)

            document = {
                "id": str(uuid.uuid4()),
                "content": chunk,
                "source": pdf_file,
                "page": page_number + 1,
                "embedding": embedding
            }

            client.collections["MDM_RAG"].documents.create(document)

print("🎉 All PDFs ingested into MDM_RAG")


100%|██████████| 6/6 [01:34<00:00, 15.75s/it]

🎉 All PDFs ingested into MDM_RAG


In [27]:
## Verifying Documents ingested
client.collections["MDM_RAG"].documents.search({
    "q": "*",
    "query_by": "content",
    "per_page": 3
})

{'facet_counts': [],
 'found': 867,
 'hits': [{'document': {'content': 'l components or the use of \nexpensive network technologies.\n• All of these need to be traded off when deciding on the best \ncombination of technologies and device specifications to use.\n2/3/2025\nDr.P.D.Khandait, HOD (E&TC),KDKCE IOT Unit-I\n117',
    'embedding': [-0.07096592336893082,
     0.00981177482753992,
     0.03673665225505829,
     -0.042998798191547394,
     0.054823119193315506,
     -0.0459323413670063,
     0.017193244770169258,
     0.081546850502491,
     0.014116168953478336,
     -0.005369171500205994,
     0.009642314165830612,
     0.021886376664042473,
     0.014448756352066994,
     -0.039911095052957535,
     0.027524879202246666,
     -0.027552485466003418,
     0.1009516790509224,
     -0.10503370314836502,
     0.003842955455183983,
     -0.0625985711812973,
     0.016457566991448402,
     0.013590320013463497,
     0.007750592660158873,
     -0.05731962248682976,
     0.0085511291399

In [30]:
search_parameters = {
    "q": "accuracy and precision",
    "query_by": "content",
    "per_page": 5
}

result = client.collections["MDM_RAG"].documents.search(search_parameters)
result


{'facet_counts': [],
 'found': 8,
 'hits': [{'document': {'content': '06-01-2026\nMDM_SENSORS_ACTUATORS\n67\n❑\nDISCUSS THE WORKING PRINCIPLE AND APPLICATIONS OF RESISTIVE, INDUCTIVE, AND \nCAPACITIVE SENSORS.\n❑\nEXPLAIN THE IMPORTANCE OF CALIBRATION IN MEASUREMENT SYSTEMS AND ITS EFFECT ON \nACCURACY AND PRECISION.\n❑\nDESCRIBE THE STATISTICAL ANALYSIS OF EXPERIMENTAL DATA, INCLUDING MEAN, DEVIATION, \nVARIANCE, AND STANDARD DEVIATION.\n❑\nJUSTIFY THE NEED FOR CALIBRATION IN MINIMIZING MEASUREMENT ERRORS.\n❑\nEXPLAIN HOW ERROR ANALYSIS AND STATISTICAL TOOLS HELP IN IMPROVING \n',
    'embedding': [0.013276705518364906,
     0.00860036164522171,
     -0.028093671426177025,
     -0.04645562171936035,
     -0.0012599426554515958,
     -0.0350133553147316,
     0.11567780375480652,
     0.06952541321516037,
     -0.05439593642950058,
     0.06537764519453049,
     0.0481044165790081,
     -0.016630830243229866,
     0.10917496681213379,
     -0.0055109248496592045,
     -0.09617390483617

{'results': [{'code': 400, 'error': 'Malformed vector query string.'}]}


In [33]:
print(client.collections["MDM_RAG"].retrieve())

{'created_at': 1769600731, 'default_sorting_field': '', 'enable_nested_fields': False, 'fields': [{'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'content', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'source', 'optional': False, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'page', 'optional': False, 'sort': True, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'int32'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'unit', 'optional': True, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'type': 'string'}, {'facet': True, 'index': True, 'infix': False, 'locale': '', 'name': 'topic', 'optional': True, 'sort': False, 'stem': False, 'stem_dictionary': '', 'store': True, 'ty